# SVI パート2： 条件付き独立、サブサンプリング、

## 目標：巨大なデータセットへの SVI のスケール

* $N$ 観測値のモデルに対して、ELBO に基づいた `model` と `guide` の推論には、$N$ に関して計算量が膨大に増える処理（対数密度関数の評価）をしなくてはならない
* スケールを目的に、モデル・ガイドに存在する条件付き独立性を活用できる仕組みがある
* 例えば、潜在変数が与えられたもとで、観測可能な変数が独立なら ELBO は次のように近似できる

$$
\sum _ { i = 1 } ^ { N } \log p \left( \mathbf { x } _ { i } | \mathbf { z } \right) \approx \frac { N } { M } \sum _ { i \in \mathcal { I } _ { M } } \log p \left( \mathbf { x } _ { i } | \mathbf { z } \right)
$$

## Pyro で条件付き独立

* 条件付き独立を指定するには `plate` か `markov` を使う

### シーケンシャル plate

```py
for i in pyro.plate("data_loop", len(data):
    pyro.sample(f"obs_{i}", dist.Bernoulli(f), obs=data[i])
```

* `pyro.plate` は組み込み関数の `range` の様に動作する
    * 一つ目の引数はユニークである必要がある
* 一つのイテレーションに対応する、`for` ループのボディ部分を **context manager** が呼ばれるもので管理する
* 時系列データなど、前後のイテレーションのデータを使わないように注意する
    * プレート表現で言い換えると、プレート内部でのループ辺は許可されない
    * こういう時は `pyro.markov` を使う

### ベクトル化 plate

```py
with pyro.plate("observe_data"):
    pyro.sample("obs", dist.Bernoulli(f), obs=data)
```

* ベクトルで表現された、複数の（互いに条件付き独立な）観測値を一気に定義したいときは ベクトル化された `plate` を用いる
* シーケンシャルの場合は各観測変数に名前をつけた（例：`obs_i`）が、ベクトル化の場合は全体に対して名前をつける
* シーケンシャルの場合の注意（例：時系列データ）は、ベクトル化 plate にも当てはまる

## サブサンプリング

* 巨大なデータセットに対する変分推定を実行するために、上で指定した条件付き独立性を元に、サブサンプリングを行う

### `plate` による自動サブサンプリング

* シーケンシャルとベクトル化で微妙に方法が違う
    * シーケンシャルの場合：`pyro.plate` のキーワード引数 `subsample_size` を指定するだけ
    * ベクトル化の場合：さらにキーワード引数 `size` を指定すれば、`plate` が抜き出すデータの索引集合を返すので、それを使う
* 使うテンソルデータが GPU 上にある場合、`device` 引数を忘れないようにする
* サブサンプリングの結果、一回も使われない変数が出てくる確率は割と高いので、サブサンプリング戦術を指定できる
    * 指定は `subsample` 引数で
    * [Primitives — Pyro documentation](http://docs.pyro.ai/en/dev/primitives.html#pyro.plate) などを参照のこと

### 局所的な潜在変数しかない時のサブサンプリング

* 潜在変数が観測変数とセットになっているときは、ミニバッチ学習で十分なのでサブサンプリングは使わない
    * $p(x,z) = \prod_n p(x_n|z_n)p(z_n) $
* 具体的には、`plate` の内部でしか `pyro.sample` が呼ばれていないようなモデルの場合
    * 例）VAE

### 局所・グローバルな潜在変数がある時のサブサンプリング

* コインフリップの例は、モデルで `plate` は使ったが、ガイドでは使っていなかった
    * サブサンプルの対象になり得るのは観測値に関するところだけだったから
* モデルとガイドの両方に `plate` が出てくる場合は？
    * ガイドのほうにだけ `subsample_size` を指定する
    * これは、サブサンプルする索引集合はモデルとガイドで同じであり、学習の最後までずっと同じだから
    * このような場合の例は [SVI Part II: Conditional Independence, Subsampling, and Amortization — Pyro Tutorials 0.3.0 documentation](http://pyro.ai/examples/svi_part_ii.html) を参照
    
## Amortization

* 次のような変分分布を考える。
$$
q ( \mathbf { z } , \beta ) = q ( \beta ) \prod _ { i = 1 } ^ { N } q \left( \mathbf { z } _ { i } | \beta , \lambda _ { i } \right)
$$
* 各観測値 $x_i$ に変分パラメータ $\lambda_i$ があるので $N$ が大きい時に最適化が難しくなる
* なので、$\lambda_i$ を代弁する関数 $\lambda(x_i, \phi)$ を学習する
    * $\phi$ は変分パラメータ
    * $N$ が増えても $\phi$ は増えないので、データ数に対してスケール可能！
* このトリックを使ったSVIを**amortized variational inference**という
    * 変分オートエンコーダーで使われているテクニック